In [ ]:
# Importando as bibliotecas necessárias:
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd

# Importandos as bibliotecas do Scikit Learn:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import LeavePOut

# Usando função mágica para plotagem:
%matplotlib inline

In [ ]:
# Importando os dados do csv:
df = pd.read_csv('sum_cosines.csv', header=None)

# Definindo a quantidade de amostras:
M = len(df[0].to_numpy())

# Definindo os valores das features:
x = (df[0].to_numpy()).reshape(M, 1)

# Definindo os valores dos targets:
y = (df[1].to_numpy()).reshape(M, 1)

In [ ]:
# Plotando a figura:
fig = plt.figure(figsize=(10, 10))
plt.plot(x, y, 'b.')
plt.show()

# Holdout

In [ ]:
# Definindo o tamanho do conjunto de validação:
size = 0.3

# Aplicando o HOLDOUT:
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=size)

In [ ]:
# Imprimindo a segmentação dos conjuntos:
print('{} das valores estão separado para o conjunto de treinamento enquanto {} estão separados para o conjunto de validação'.format(len(x_train), len(x_valid)))

In [ ]:
# Definição do BIAS:
bias = True

# Vetor de predição:
mean_vec = []
# Vetor de treinamento:
train_vec = []

for d in range(1, 21):
    
    # Instanciando o Polynomial Features
    poly_features = PolynomialFeatures(degree=d, include_bias=bias)
    # Instanciando a padronização:
    std_scaler = StandardScaler()
    # Tnstanciando aRegressão Linear
    lin_reg = LinearRegression()
    
    # Definindo a sequência de ações:
    polynomial_regression = Pipeline([
        ("poly_features", poly_features),
        ("std_scaler", std_scaler),
        ("lin_reg", lin_reg),
    ])
    
    polynomial_regression.fit(x_train, y_train)
    
    y_train_predict = polynomial_regression.predict(x_train)
    
    y_valid_predict = polynomial_regression.predict(x_valid)
    
    train_vec.append(np.sqrt(mean_squared_error(y_train, y_train_predict)))
    
    mean_vec.append(np.sqrt(mean_squared_error(y_valid, y_valid_predict)))   

In [ ]:
# Plotando grafico:
plt.figure(figsize=(7, 7))
plt.plot(range(1, 21), train_vec, label='Treinamento')
plt.plot(range(1, 21), mean_vec, label='Validação')
plt.xlabel('Polinômio', fontsize=16)
plt.ylabel('EQM', fontsize=16)
plt.xticks(range(0, 22, 2))
plt.xlim([1, 22])
plt.legend()
plt.grid()
plt.show()

# K fold

In [ ]:
# Definindo o valor de K:
K = 10

# Aplicando:
kfold = KFold(n_splits=K, shuffle=True)

In [ ]:
# Definição do BIAS:
bias = True

# Vetor de predição:
mean_vec = []
# Vetor de padronização:
std_vec = []

for d in range(1, 21):
    
    # Instanciando o Polynomial Features
    poly_features = PolynomialFeatures(degree=d, include_bias=bias)
    # Instanciando a padronização:
    std_scaler = StandardScaler()
    # Tnstanciando aRegressão Linear
    lin_reg = LinearRegression()
    
    # Definindo a sequência de ações:
    polynomial_regression = Pipeline([
        ("poly_features", poly_features),
        ("std_scaler", std_scaler),
        ("lin_reg", lin_reg),
    ])
    
    lin_scores = cross_val_score(polynomial_regression, x, y, scoring='neg_mean_squared_error', cv=kfold) 
        
    scores = np.sqrt(-lin_scores)
    mean_vec.append(scores.mean())
    std_vec.append(scores.std())

In [ ]:
# Plotando os gráficos

plt.figure(figsize=(20, 5))
ax = plt.subplot(1, 2, 1)
plt.plot(range(1, 21), mean_vec,  label='EQM')
plt.xlabel('Ordem do Polinômio', fontsize=16)
plt.ylabel('EQM', fontsize=16)
plt.xticks(range(0, 22, 2))
plt.xlim([1, 22])
plt.legend()
plt.grid()

ax = plt.subplot(1, 2, 2)
plt.plot(range(1, 21), std_vec,  label='Desvio Padrão')
plt.xlabel('Ordem do Polinômio', fontsize=16)
plt.ylabel('Desvio Padrão', fontsize=16)
plt.xticks(range(0, 22, 2))
plt.xlim([1, 22])
plt.legend()
plt.grid()
plt.show()

# Leave-p-out

In [ ]:
# Vetor de predição:
mean_vec = []
# Vetor de padronização:
std_vec = []

# Instanciando o Leave-p-out
loocv = LeaveOneOut()

for d in range(1, 21):
    
    # Instanciando o Polynomial Features
    poly_features = PolynomialFeatures(degree=d, include_bias=bias)
    # Instanciando a padronização:
    std_scaler = StandardScaler()
    # Tnstanciando aRegressão Linear
    lin_reg = LinearRegression()
    
    # Definindo a sequência de ações:
    polynomial_regression = Pipeline([
        ("poly_features", poly_features),
        ("std_scaler", std_scaler),
        ("lin_reg", lin_reg),
    ])
    
    lin_scores = cross_val_score(polynomial_regression, x, y, scoring='neg_mean_squared_error', cv=loocv) 
        
    scores = np.sqrt(-lin_scores)
    mean_vec.append(scores.mean())
    std_vec.append(scores.std())

In [ ]:
# Plotando os gráficos

plt.figure(figsize=(20, 5))
ax = plt.subplot(1, 2, 1)
plt.plot(range(1, 21), mean_vec,  label='EQM')
plt.xlabel('Polinômio', fontsize=16)
plt.ylabel('Média EQM', fontsize=16)
plt.yscale('log')
plt.xticks(range(0, 22, 2))
plt.xlim([1, 22])
plt.legend()
plt.grid()

ax = plt.subplot(1, 2, 2)
plt.plot(range(1, 21), std_vec,  label='Desvio Padrão')
plt.xlabel('Polinômio', fontsize=16)
plt.ylabel('Desvio Padrão', fontsize=16)
plt.xticks(range(0, 22, 2))
plt.xlim([1, 22])
plt.legend()
plt.grid()

# Curva de aprendizado

In [ ]:
# Definindo os graus:
degrees = [1,3,5,20]

# Para plotagem das figuras:
plt.figure(figsize=(20, 5))

# LOOP
for i in range(len(degrees)):
    ax = plt.subplot(1, len(degrees), i + 1)

    # Insanciando o Polynomial Features
    poly_features = PolynomialFeatures(degree=degrees[i], include_bias=include_bias)
    # Instanciando o Padronizados
    std_scaler = StandardScaler()
    # Instanciando a Regressão Linear
    lin_reg = LinearRegression()

    # Definindo a sequencia de ações:
    polynomial_regression = Pipeline([
        ("poly_features", poly_features),
        ("std_scaler", std_scaler),
        ("lin_reg", lin_reg),
    ])
    
    # Criando o vetor de armanezamento dos erros de treinamento:
    train_error = []
    # Criando o vetor de armazenamento dos erros de validação:
    valid_error = []
    
    # LOOP
    for m in range(1,len(x_treino)+1):

        
        polynomial_regression.fit(x_train[:m], y_train[:m])
        
        y_train_predict = polynomial_regression.predict(x_train[:m])

        y_valid_predict = polynomial_regression.predict(x_valid)       

        train_error.append(mean_squared_error(y_train[:m], y_train_predict))

        valid_error.append(mean_squared_error(y_valid, y_valid_predict))
        
    plt.plot(range(1,len(x_train)+1), np.sqrt(train_error), label='Treinamento')
    plt.plot(range(1,len(x_train)+1), np.sqrt(valid_error), label='Validação')
    plt.yscale('log')
    plt.xlabel('Treinamento', fontsize=16)
    plt.ylabel('EQM', fontsize=12)
    plt.grid()
    plt.title('Polinômio de Ordem '+str(degrees[i]))
    plt.legend(loc='best')
plt.show()
   

## RESPOSTA TEÓRICA:

O algortimo parece ter uma resposta satisfatória em sua décima ordem, como no teste abaixo:

In [ ]:
# Definição do BIAS:
dg = 10

# Definição do grau do polinômio:
bias = True

# Instanciando o Polynomial Features:
poly_features = PolynomialFeatures(degree=dg, include_bias=bias)

# Instanciando o Padronizador:
std_scaler = StandardScaler()

# Instanciando a Regressão Linear:
lin_reg = LinearRegression()

# Definindo a sequencia de ações:
model = Pipeline([
    ("poly_features", poly_features),
    ("std_scaler", std_scaler),
    ("lin_reg", lin_reg),
    ])

# Fazendo o treinamento do modleo:
model.fit(x, y) 

# Validando pela predição:
predict = model.predict(x)

In [ ]:
# Plotando a comparação:
fig = plt.figure(figsize=(10,10))
plt.plot(x, y, '.', label='Versão observável')
plt.plot(x, predict, 'x', label='Modelo proposto')
plt.xlabel('$x$', fontsize=16)
plt.ylabel('$y$', fontsize=16)
plt.legend()
plt.grid()
plt.show()